# Защита персональных данных клиентов

# Описание проекта 

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Инструкция по выполнению проекта**

1. Загрузить и изучить данные.
2. Ответить на вопрос и обосновать решение. 
- Признаки умножают на обратимую матрицу. 
- Изменится ли качество линейной регрессии? (Её можно обучить заново.)
- a. Изменится. Приведите примеры матриц.
- b. Не изменится. Указать, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
4. Предложить алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.
5. Запрограммировать этот алгоритм, применив матричные операции. Проверить, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применить метрику R2.

**Описание данных**

Набор данных находится в файле /datasets/insurance.csv.<br>
__Признаки__: пол, возраст и зарплата застрахованного, количество членов его семьи.<br>
__Целевой признак__: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
# Импортируем библиотеку для общего удобства
import pandas as pd
import numpy as np

from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [2]:
try:
    data = pd.read_csv('/datasets/insurance.csv')
except:
    data = pd.read_csv('insurance.csv')
data.name = 'Данные клиентов'

In [3]:
# Функция для отображения общей информации
def review(df):
    info = \
    print(f'                 >{df.name}<'),'\n',
    display(df.head()),
    print(),
    df.info(),
    print()
    print('-' * 40, '\n', 
          'Количество дубликатов:',df.duplicated().sum(),
          '\n')

In [4]:
review(data)

                 >Данные клиентов<


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB

---------------------------------------- 
 Количество дубликатов: 153 



In [5]:
# Убере дублирующие строки для точности матрицы
data = data.drop_duplicates().reset_index(drop=True)
# А так же заменим тип данных Возраст, Зарплата на int
data.Возраст = data.Возраст.astype('int64')
data.Зарплата = data.Зарплата.astype('int64')
data.info()
data.describe()
print()
print(f'''Проверим на пропуски 
{data.isna().sum()}''')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                4847 non-null   int64
 1   Возраст            4847 non-null   int64
 2   Зарплата           4847 non-null   int64
 3   Члены семьи        4847 non-null   int64
 4   Страховые выплаты  4847 non-null   int64
dtypes: int64(5)
memory usage: 189.5 KB

Проверим на пропуски 
Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64


In [6]:
data.columns = ['male', 'age', 'salary', 'family_members', 'insurance_payment']
data.head()

,male,age,salary,family_members,insurance_payment
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

In [7]:
# Готовим целевой признак и признаки
features=data.drop(['insurance_payment'],axis=1)
target=data['insurance_payment']

In [8]:
print(features.shape, end ='\n')
print(target.shape)

(4847, 4)
(4847,)


In [9]:
random_matrix = np.random.randn(features.shape[1],features.shape[1])

Проверим матрицу на обратимость.

In [10]:
try:
    inv_matrix = np.linalg.inv(random_matrix)
    print('\nВывод: \nОбратная матрица к заданной существует')
except:
    print('\nВывод: Обратная матрица к заданной не существует')


Вывод: 
Обратная матрица к заданной существует


In [11]:
a = np.random.randn(features.shape[1],features.shape[1])
a_inv = np.linalg.inv(a)
print(a_inv)
print('\n',np.dot(a, a_inv))

[[-0.25530302 -0.47285476  0.70726904  0.17747791]
 [-0.08727969  0.39751579 -1.15192753 -0.7782626 ]
 [-0.42209161  0.05487826  0.32510495  0.52159973]
 [ 0.46940267  0.05874443  1.24310246  0.15674822]]

 [[ 1.00000000e+00 -1.89672847e-17  1.47359848e-16  8.40002463e-17]
 [-1.97387949e-17  1.00000000e+00 -6.06085523e-16 -2.54041678e-16]
 [ 4.27323610e-17  4.01231603e-17  1.00000000e+00 -3.78530403e-17]
 [-6.90104713e-18 -3.68020368e-17 -4.74557302e-17  1.00000000e+00]]


Мы можем проверить правильность результата, опираясь на основное свойство обратной матрицы, т.е.

In [12]:
print(np.allclose(np.dot(a, a_inv), np.eye(a.shape[0])))
print()
print(np.allclose(np.dot(a_inv, a), np.eye(a.shape[0])))


True

True


Рассчитаем результаты для исходных параметров.

In [13]:
w = np.linalg.inv(features.T.dot(features)).dot(features.T).dot(target)
a = features @ w

Рассчитаем результаты для новых параметров.

In [14]:
features_upgraded = features @ random_matrix

In [15]:
w2 = np.linalg.inv(features_upgraded.T.dot(features_upgraded)).dot(features_upgraded.T).dot(target)
a2 = features_upgraded @ w2

Посмотрим, есть ли разница между результатами.

In [16]:
difference = a - a2
difference.sum()

0.0005379558330596888

**Вывод:**
Используемый алгоритм не создает различий в предсказаниях при использовании исходных параметров и модернизированных. Значит, качество предсказаний модели не ухудшится.

Разница между исходными предсказаниями и предсказаниями с использованием обратимой матрицы не значительно минимально.

**Ответ:** при умножении признаков $X$
 на обратимую матрицу $P$
 мы имеем новую матрицу признаков $B$
. Качество линейной регрессии после проверки $B$
 измениться, т.е метрика качества $R^2$
 будет сильно другой. А точнее будет хуже. Но если принять матрицу признаков $B$
, как обучающую, то модель обучится и покажет тот же результат $R^2$
, что и для признаков $X$


**Обоснование:**

Используемые свойства:
$$
(AB)^T=B^T A^T
$$
$$
(AB)^{-1} = B^{-1} A^{-1}
$$
$$
A A^{-1} = A^{-1} A = E
$$
$$
AE = EA = A
$$
Доказательство:
$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$
\
Требуется доказать, что предсказания не изменятся, имеем  $a =  Xw$,   $a' = X'w'$
\
\
$$
    w = (X^T X)^{-1} X^T y
$$

$$  
w' = ((XP)^TXP)^{-1}(XP)^Ty = (P^T(X^TX)P)^{-1}P^TX^Ty = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty = P^{-1}(X^TX)^{-1}X^Ty => w' = P^{-1}w
$$

$$ a = Xw$$

Получаем: $a'= XP * w' = XPP^{-1}w = XEw = Xw = a$

Таким образом мы доказали, что преобразование подобного типа не повлияет на предсказание.

## Алгоритм преобразования

**Алгоритм**

После получения новой матрицы B, перед процессом функцией predict выполним действие: $X = P^-¹ * B$

**Обоснование**

Кодирование признаков происходит умножением $X * P = B$,<br>
где: <br>
$X$-матрица признаков,
<br>$P$ 
-матрица, на которую умножаются признаки (обратимая матрица),
<br>$B$
-новая матрица. <br>Умножим обе части уравнения на $P^-¹$
 (обратную матрицу $P$
). <br>Получим $X * P^-¹ * P = B * P^-¹$
. <br>Зная свойство обратных матриц $P^-¹ * P = E$
, где Е- это единичная матрица и зная свойство единичной матрицы $E * X = X$ 
, получаем, что $X = P^-¹ * B$

## Проверка алгоритма

In [17]:
#Создадим класс модели Линейной Регрессии.
class ConstantRegression:
    def model_fit(self, features, target):
        self.model = LinearRegression()
        self.model.fit(features, target)
        
    def predict_result(self, features):
        result = self.model.predict(features)
        return result
        
    def get_r2_s(self, target, predict):
        r2 = r2_score(target, predict)
        return r2  

In [18]:
#Вычислим результат метрики для исходных признаков.
model_1 = ConstantRegression()
model_1.model_fit(features, target)
predict_features = model_1.predict_result(features)
r2_features = model_1.get_r2_s(target, predict_features)

In [19]:
#Вычислим результат метрики для признаков, умноженных на обратимую матрицу.
model_2 = ConstantRegression()
model_2.model_fit(features_upgraded, target)
predict_upgraded_features = model_2.predict_result(features_upgraded)
r2_upgraded_features = model_2.get_r2_s(target, predict_upgraded_features)

In [20]:
print('\nЗначение R2 при использовании исходных признаков -', r2_features)
print('\nЗначение R2 при использовании признаков, умноженых на обратимую матрицу -', r2_upgraded_features)


Значение R2 при использовании исходных признаков - 0.4302010046633359

Значение R2 при использовании признаков, умноженых на обратимую матрицу - 0.43020100466332967


**Вывод:** 

Разница не значительно минимальна, при умножении признаков на обратимую матрицу качество модели не изменилось.

**Вывод**

- Используемый алгоритм не создает различий в предсказаниях при использовании исходных параметров и модернизированных. Значит, качество предсказаний модели не ухудшится.

- При умножение матрицы признаков на обратимую матрицу качество Линейной Регрессии не изменяется.